# Predição final para validação do modelo

In [22]:
import pandas as pd
import pickle 
import csv
import numpy as np

In [64]:
lexicon = np.loadtxt('csvs/lexicon_enriquecido.csv', skiprows=1, dtype='str', delimiter='\n')

def classificar_base(base):

    with open('model.pkl', 'rb') as f:
        model = pickle.load(f)
        classes = model.predict(base)
    return classes

In [93]:
base = pd.read_csv('csvs/processados/tweets_odio_lula_processado.csv')

In [95]:
result = classificar_base(base_features)

In [94]:
from sklearn.feature_extraction.text import CountVectorizer
with open('vect.pkl', 'rb') as f:
        vect = pickle.load(f)

base_features = vect.transform(base.text)

In [96]:
teste = result[:100]

In [97]:
teste2 = pd.read_csv('csvs/processados/odio_manual_lula.csv', usecols=['classe'])

In [98]:
from sklearn.metrics import accuracy_score
accuracy_score(teste2,teste)

0.78